In [ ]:
import yfinance as yf
from stocktrends import Renko

In [ ]:
tickers = ["MSFT","AMZN","GOOG"]
ohlcv_data={}
hour_data={}
renko_data={}

In [ ]:
for ticker in tickers:
    temp = yf.download(ticker,period='1mo',intervals='5m')
    temp.dropna(how="any",inplace=True)
    ohlcv_data[ticker]=temp
    temp = yf.download(ticker,period='1y',intervals='1h')
    temp.dropna(how="any",inplace=True)
    hour_data[ticker]=temp

    def ATR(DF,n=14):
            df=DF.copy()
            df["H-L"] = df["High"]-df["Low"]
            df["H-PC"] = df["High"] - df["Adj Close"].shift(1)
            df["L-PC"] = df["Low"] - df["Adj Close"].shift(1)
            df["TR"] = df[["H-L","H-PC","L-PC"]].max(axis=1,skipna=False)
            df["ATR"]=df["TR"].ewm(com=n,min_periods=n).mean() #com gets you close to yahoo and span gets you close to traderview
            return df["ATR"]

In [ ]:
def renko_DF(DF,hourly_df):
    df = DF.copy()
    df.drop("Close",axis=1,inplace=True)
    df.reset_index(inplace=True)
    df.columns = ["date","open","high","low","close","volume"]
    df2 = Renko(df)
    df2.brick_size = 3*round(ATR(hourly_df,120).iloc[-1],0)
    renko_df=df.get_ohlc_data()
    return renko_df

In [ ]:
for ticker in ohlcv_data:
    renko_data[ticker] = renko_DF(ohlcv_data[ticker],hour_data[ticker])